#The Modern Mamak: Estimating Date of Termination of Non-live Shops by Proxy

Unfortunately, ACRA does not provide dates of termination of a registered business in Singapore. However, its registry does have records of other businesses and their addresses. In order to estimate the date of a shop's termination by proxy, the dataframe of non-live shops will be compared against the the dataframe of ACRA's entire registry to find any matching addresses. It is assumed that when a shop closes at one location, another often registers and takes over the rental space shortly after. Some issues that may arise could be:

* A location may have remained vacant for several years before being taken over by another business *(inaccurate termination date)*
* A location may have never been taken over by another business since then *(possible null value for termination date)*
* A location may have been removed completely *(possible null value for termination date)*
* A location may have been repurposed for non-business activities *(possible null value for termination date)*

<br>

Although this method is rather inaccurate, the project requires mostly broadstrokes analyses to proceed with the research. Additionally, other business directories online were scraped to determine some estimated termination dates. These directories, however, do not publish a business' termination date but rather the last update the directory has made to the status based on ACRA data. These directories are SGBusiness and RecordOwl. They will be compared to these estimated dates of termination and the earliest date would be the most likely date of termination.

# Libraries

In [ ]:
#Google credentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapefile as shp

# Read Files

In [ ]:
file_path = '/content/drive/MyDrive/Mamak Stores/ACRA data/all_acra_businesses.csv'
comb_df = pd.read_csv(file_path)

<ipython-input-3-b66bb05416f5>:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  comb_df = pd.read_csv(file_path)


In [ ]:
path_b = "/content/drive/MyDrive/Mamak Stores/ACRA data/mamaks_b1980s_geo.csv"
mamaks_b1980s = pd.read_csv(path_b)

In [ ]:
mamaks_b1980s['entity_status_description'].head(100)

,entity_status_description
0,na
1,na
2,na
3,na
4,na
...,...
95,na
96,Cancelled (Non-Renewal)
97,na
98,na


# Combine All Non-live Shops

In [ ]:
mamaks_b1980s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2317 entries, 0 to 2316
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   uen                                2317 non-null   object 
 1   issuance_agency_id                 2317 non-null   object 
 2   entity_name                        2317 non-null   object 
 3   entity_type_description            2317 non-null   object 
 4   business_constitution_description  2317 non-null   object 
 5   company_type_description           2317 non-null   object 
 6   paf_constitution_description       2317 non-null   object 
 7   entity_status_description          2317 non-null   object 
 8   registration_incorporation_date    2317 non-null   object 
 9   uen_issue_date                     2317 non-null   object 
 10  address_type                       2317 non-null   object 
 11  block                              2317 non-null   objec

In [ ]:
mamaks_b1980s.head()

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,primary_ssic_code,primary_ssic_description,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity,registration_year,Address,Add_lat,Add_lon
0,03056700M,ACRA,WING THYE HUP KEE,Business,Sole Proprietor,na,na,na,1974-12-09,2008-09-09,...,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,127 KALLANG ROAD SINGAPORE 208706,1.308109,103.863335
1,03091500K,ACRA,WING HENG & COMPANY,Business,Partnership,na,na,na,1974-11-26,2008-09-09,...,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1.281754,103.844879
2,03143800L,ACRA,WIN & COMPANY,Business,Sole Proprietor,na,na,na,1974-12-07,2008-09-09,...,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,1.311125,103.788211
3,03389400X,ACRA,WAN LEE,Business,Sole Proprietor,na,na,na,1974-12-11,2008-09-09,...,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,128 ST. JOHN'S ROAD SINGAPORE 2775,1.456403,103.827852
4,03496500M,ACRA,W.S. RAM SINGH CO,Business,Sole Proprietor,na,na,na,1974-12-28,2008-09-09,...,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,242 SERANGOON ROAD SINGAPORE 218087,1.309940,103.854199


In [ ]:
cols_1 = ['block', 'street_name']
cols_2 = ['building_name']

#address_1 (block + street_name)
address_1 = (
    comb_df[cols_1]
    .fillna('')
    .astype(str)
    .replace('na', '')
    .agg(' '.join, axis=1)
    .str.strip()
)

#'#level_no-unit_no' (if both exist)
level_unit = comb_df[['level_no', 'unit_no']].fillna('').astype(str).replace('na', '')
address_2 = level_unit.apply(
    lambda x: f"#{x.level_no}-{x.unit_no}" if x.level_no and x.unit_no
    else f"#{x.unit_no}" if x.unit_no
    else '',
    axis=1
)

#combine with building_name
address_2 += ' ' + (
    comb_df[cols_2]
    .fillna('')
    .astype(str)
    .replace('na', '')
    .agg(' '.join, axis=1)
    .str.strip()
)

#add 'SINGAPORE' before postal code
address_3 = comb_df['postal_code'].fillna('').astype(str).replace('na', '').apply(
    lambda x: f"SINGAPORE {x}"
)

#full address
comb_df['Address'] = (
    address_1 + ' ' + address_2.str.strip() + ' ' + address_3
).str.replace(r'\s+', ' ', regex=True).str.strip()


#Find Businesses that Replaced Non-live Shops

The entities listed in the dataframe of non-live shops are eliminated to avoid confusion with replaced shops. Then this dataframe of all other businesses will be checked for the same address to confirm that a mamak shop has been replaced. This dataset of replaced shops may contain the subsequent businesses that have also replaced those that initially replaced the mamak shops. Hence, the business with the registration date closest to that of the mamak shop will be used.

In [ ]:
comb_df['entity_status_description'] = comb_df['entity_status_description'].astype(str)
comb_df_2 = comb_df.copy()
comb_df_2 = comb_df[~comb_df['entity_name'].isin(mamaks_b1980s['entity_name'])]

In [ ]:
comb_df_2.head(3)

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,name_of_audit_firm1,uen_of_audit_firm2,name_of_audit_firm2,uen_of_audit_firm3,name_of_audit_firm3,uen_of_audit_firm4,name_of_audit_firm4,uen_of_audit_firm5,name_of_audit_firm5,Address
0,00012700X,ACRA,WAH SEONG PRESS,Business,Sole Proprietor,na,na,Ceased Registration,1924-01-01,2008-09-09,...,na,na,na,na,na,na,na,na,na,5 KUNG CHONG ROAD SINGAPORE 159142
1,00021600C,ACRA,WONG YIN FOH,Business,Sole Proprietor,na,na,Cancelled,1974-09-19,2008-09-09,...,na,na,na,na,na,na,na,na,na,259 VICTORIA STREET SINGAPORE 0718
2,00036800L,ACRA,WOON BROTHERS CO,Business,Partnership,na,na,na,1974-09-28,2008-09-09,...,na,na,na,na,na,na,na,na,na,101 KITCHENER ROAD #03-24 JALAN BESAR PLAZA SI...


In [ ]:
replaced_shops = comb_df_2[comb_df_2['Address'].isin(mamaks_b1980s['Address'])]

In [ ]:
replaced_shops.head()

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,name_of_audit_firm1,uen_of_audit_firm2,name_of_audit_firm2,uen_of_audit_firm3,name_of_audit_firm3,uen_of_audit_firm4,name_of_audit_firm4,uen_of_audit_firm5,name_of_audit_firm5,Address
63,00854500J,ACRA,WAH KEE,Business,Sole Proprietor,na,na,na,1974-10-24,2008-09-09,...,na,na,na,na,na,na,na,na,na,39 PURVIS STREET SINGAPORE 188616
71,00949600M,ACRA,WAH KIEW,Business,Partnership,na,na,na,1974-12-04,2008-09-09,...,na,na,na,na,na,na,na,na,na,460 RACE COURSE ROAD SINGAPORE 218700
79,01016000X,ACRA,WONG TAT KEE,Business,Sole Proprietor,na,na,Cancelled,1974-10-11,2008-09-09,...,na,na,na,na,na,na,na,na,na,52 SMITH STREET SINGAPORE 058959
98,01632900K,ACRA,WAH HONG,Business,Partnership,na,na,Live,1974-10-16,2008-09-09,...,na,na,na,na,na,na,na,na,na,586 GEYLANG ROAD SINGAPORE 389525
195,04240400B,ACRA,WINGS & COMPANY,Business,Sole Proprietor,na,na,Cancelled,1974-12-21,2008-09-09,...,na,na,na,na,na,na,na,na,na,6 DE SOUZA STREET SINGAPORE 0104


In [ ]:
# 1: Merge provision shops with all businesses that share the same address
merged = mamaks_b1980s.merge(
    comb_df,
    on='Address',
    how='left',
    suffixes=('_provision', '_nextshops')
)

# 2: Keep only businesses that opened AFTER the provision shop
merged = merged[merged['registration_incorporation_date_nextshops'] > merged['registration_incorporation_date_provision']]

# 3: Find the next closest registration date per address
next_business = merged.groupby('Address')['registration_incorporation_date_nextshops'].min().reset_index()

# 4: Merge back to `mm_all_nlive` to assign estimated `tdate_proxy`
mamaks_b1980s = mamaks_b1980s.merge(next_business, on='Address', how='left')
mamaks_b1980s.rename(columns={'registration_incorporation_date_nextshops': 'tdate_proxy'}, inplace=True)

mamaks_b1980s

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,primary_ssic_description,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity,registration_year,Address,Add_lat,Add_lon,tdate_proxy
0,03056700M,ACRA,WING THYE HUP KEE,Business,Sole Proprietor,na,na,na,1974-12-09,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,127 KALLANG ROAD SINGAPORE 208706,1.308109,103.863335,1990-04-06
1,03091500K,ACRA,WING HENG & COMPANY,Business,Partnership,na,na,na,1974-11-26,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1.281754,103.844879,1992-01-28
2,03143800L,ACRA,WIN & COMPANY,Business,Sole Proprietor,na,na,na,1974-12-07,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,1.311125,103.788211,NaN
3,03389400X,ACRA,WAN LEE,Business,Sole Proprietor,na,na,na,1974-12-11,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,128 ST. JOHN'S ROAD SINGAPORE 2775,1.456403,103.827852,NaN
4,03496500M,ACRA,W.S. RAM SINGH CO,Business,Sole Proprietor,na,na,na,1974-12-28,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1974,242 SERANGOON ROAD SINGAPORE 218087,1.309940,103.854199,1975-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,21963600A,ACRA,ELLENBOROUGH STREET GUAN SOON,Business,Partnership,na,na,na,1976-07-30,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1976,3 #03-131 SINGAPORE 050003,1.352083,103.819836,NaN
2313,22004600W,ACRA,ELLENBOROUGH NGEE HENG,Business,Partnership,na,na,Ceased Registration,1976-08-13,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1976,108 HOUGANG AVENUE 1 #01-1323 SINGAPORE 530108,1.353421,103.890164,1982-11-24
2314,22106500L,ACRA,EUNOS CRESCENT MINI MART,Business,Sole Proprietor,na,na,na,1976-09-20,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1976,6 EUNOS CRESCENT #01-2633 SINGAPORE 400006,1.321170,103.904285,1997-12-29
2315,23622300C,ACRA,ENG CHOON PROVISION SHOP,Business,Sole Proprietor,na,na,na,1978-02-21,2008-09-09,...,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,1978,3V LORONG RENJONG SINGAPORE 1954,1.347137,103.860086,NaN


In [ ]:
next_business

,Address,registration_incorporation_date_nextshops
0,1 BENCOOLEN STREET SINGAPORE 0718,1979-09-04
1,1 COLOMBO COURT #06-03 SINGAPORE 179742,1979-12-05
2,1 COLOMBO COURT #06-24B SINGAPORE 179742,1988-11-25
3,1 HORNE ROAD SINGAPORE 0820,1977-01-22
4,1 JALAN BERSEH SINGAPORE 209037,1976-03-25
...,...,...
1214,98A ST. JOHN'S ROAD SINGAPORE 2775,1984-02-27
1215,A WATERLOO STREET SINGAPORE 0718,1982-12-07
1216,CARPENTER STREET #02-18 SINGAPORE 0105,1986-09-29
1217,PRINCE EDWARD ROAD FINGER PIER BUILDING SINGAP...,1987-05-13


In [ ]:
termination_data_proxy = mamaks_b1980s[['entity_name', 'Address', 'tdate_proxy']]
termination_data_proxy.head()

,entity_name,Address,tdate_proxy
0,WING THYE HUP KEE,127 KALLANG ROAD SINGAPORE 208706,1990-04-06
1,WING HENG & COMPANY,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1992-01-28
2,WIN & COMPANY,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,NaN
3,WAN LEE,128 ST. JOHN'S ROAD SINGAPORE 2775,NaN
4,W.S. RAM SINGH CO,242 SERANGOON ROAD SINGAPORE 218087,1975-04-04


In [ ]:
termination_data_proxy['tdate_proxy'] = pd.to_datetime(
    termination_data_proxy['tdate_proxy'], errors='coerce')

termination_data_proxy['termination_year'] = termination_data_proxy['tdate_proxy'].dt.year

<ipython-input-25-c11112d92779>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  termination_data_proxy['tdate_proxy'] = pd.to_datetime(
<ipython-input-25-c11112d92779>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  termination_data_proxy['termination_year'] = termination_data_proxy['tdate_proxy'].dt.year


In [ ]:
termination_data_proxy.head()

,entity_name,Address,tdate_proxy,termination_year
0,WING THYE HUP KEE,127 KALLANG ROAD SINGAPORE 208706,1990-04-06,1990.0
1,WING HENG & COMPANY,272 SOUTH BRIDGE ROAD SINGAPORE 058821,1992-01-28,1992.0
2,WIN & COMPANY,BLK 20 GHIM MOH ROAD STALL 157 SINGAPORE 270020,NaT,NaN
3,WAN LEE,128 ST. JOHN'S ROAD SINGAPORE 2775,NaT,NaN
4,W.S. RAM SINGH CO,242 SERANGOON ROAD SINGAPORE 218087,1975-04-04,1975.0


In [ ]:
#SAVE
termination_data_proxy.to_csv('/content/drive/MyDrive/Mamak Stores/shop_termination_data/termination_data_proxy_b.csv', index=False)

There are about 4516 terminated mamak shops since the 1980s. With this method, only about 2167 matching addresses were found from the rest of the ACRA database. This means over half of these locations were either never taken over, replaced by non-business activities or removed entirely. The next part of the analysis will consist of preparing data for URL webscraping and combining these estimated dates to compare and clean the data for further analysis.